In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import ndimage
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, accuracy_score, recall_score,precision_score
from sklearn.preprocessing import StandardScaler, normalize

### On active l'utilisation du GPU pour accélérer l'entrainement des modèles

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy()

### On récupère les données

In [ ]:
test_data = pd.read_csv('./data/exoTest.csv')
train_data = pd.read_csv('./data/exoTrain.csv')

### Exploration des données

#### Aperçu des données

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
columns = list(train_data.columns)

#### On observe la répartition des données

In [ ]:
plt.figure(figsize=(4,8))
colors = ["0", "1"]
sns.countplot(x='LABEL', data=train_data, palette = "Set2")
print(train_data['LABEL'].value_counts())
plt.title('Stars Distribution \n (0: Etoiles sans exoplanètes || 1: Etoiles avec exoplanètes)', fontsize=14)

#### Les données sont nettoyées mais on vérifie qu'il n'y a pas de données manquantes au cas où

In [ ]:
print("Nombre de valeurs nulles : " + str(train_data.isnull().sum().sum()))

### Affichage des corrélations

In [ ]:
# corrMatrix = train_data.corr()
# corrMatrix['LABEL'].sort_values(ascending=False) 

In [ ]:
# plt.figure(figsize=(15,15))
# sns.heatmap(train_data.corr())
# plt.title('Correlation in the data')
# plt.show()

### On sait que les étoiles ayant potentiellement des exoplanètes en orbites autour d'elles subissent une variation de leur "flux", de leur intensité lumineuse

##### Commençons avec les étoiles ne possédant pas d'exploplanetes

In [ ]:
train_data.loc[train_data['LABEL'] == 1]

##### On sélectionne 3 étoiles parmi les concernées (ici la ligne 37,2500 et 5086)

In [ ]:
# labels_1=[37,2500,5086]


fig, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].hist(train_data.iloc[37,:], bins=200)
ax[1].hist(train_data.iloc[2500,:], bins=200)
ax[2].hist(train_data.iloc[5086,:], bins=200)

ax[0].set_xlabel("Valeur des flux")
ax[1].set_xlabel("Valeur des flux")
ax[2].set_xlabel("Valeur des flux")
fig.suptitle("Répartition de la valeur des flux")

plt.show()

##### On remarque que l'intensité lumineuse de ces étoiles ne varie pas ou très peu

#### Au tour des étoiles dont on sait qu'elles possèdent des exoplanetes en orbite

In [ ]:
train_data.loc[train_data['LABEL'] == 2]

##### On sélectionne 3 étoiles parmi les concernées (ici la ligne 0,26 et 34)

In [ ]:
# labels_1=[34,26,0]

fig, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].hist(train_data.iloc[34,:], bins=200)
ax[1].hist(train_data.iloc[26,:], bins=200)
ax[2].hist(train_data.iloc[0,:], bins=200)

ax[0].set_xlabel("Valeur des flux")
ax[1].set_xlabel("Valeur des flux")
ax[2].set_xlabel("Valeur des flux")
fig.suptitle("Répartition de la valeur des flux")

plt.show()

#### A l'inverses des étoiles n'ayant pas d'exoplanètes, on observe ici une importante variation de l'intensité lumineuse

### Préparation des données

#### On convertit les valeurs du Label en binaire pour faciliter le traitement des données

In [ ]:
label_change = {1: 0,2: 1}
train_data.LABEL = [label_change[item] for item in train_data.LABEL]
test_data.LABEL = [label_change[item] for item in test_data.LABEL]

In [ ]:
train_data

In [ ]:
test_data

### On a pu remarquer des anomalies dans les données (les flux), on peut donc essayer de visualiser cela avec des boites à moustache

In [ ]:
fig, axes = plt.subplots(1, 5,figsize=(15, 6), sharey=True)
fig.suptitle('Distribution of FLUX')

sns.boxplot(ax=axes[0], data=train_data, x='LABEL', y='FLUX.1',palette="Set2")
sns.boxplot(ax=axes[1], data=train_data, x='LABEL', y='FLUX.2',palette="Set2")
sns.boxplot(ax=axes[2], data=train_data, x='LABEL', y='FLUX.3',palette="Set2")
sns.boxplot(ax=axes[3], data=train_data, x='LABEL', y='FLUX.4',palette="Set2")
sns.boxplot(ax=axes[4], data=train_data, x='LABEL', y='FLUX.5',palette="Set2")

#### On remarque donc des anomalies (des outliers), notamment pour les étoiles ne possédant pas d'exoplanetes, que l'on va retirer car ils peuvent perturber l'apprentissage du model plus tard

#### Retrait des anomalies (outliers)

In [ ]:
# def remove_outliers(df):
#     for col in df.columns:
#         print("capping the ",col)
#         if (col != 'LABEL'):
#             percentiles = df[col].quantile([0.02,0.98]).values
#             df[col][df[col] <= percentiles[0]] = percentiles[0]
#             df[col][df[col] >= percentiles[1]] = percentiles[1]
#         else:
#             df[col]=df[col]
#     return df
    

# # final = remove_outliers(train_data)

##### on retire les 2% des valeurs les plus hautes et les plus basses pour les remplacer par les valeurs les plus proches

In [ ]:
fig, axes = plt.subplots(1, 5,figsize=(15, 6), sharey=True)
fig.suptitle('Distribution of FLUX')

sns.boxplot(ax=axes[0], data=train_data, x='LABEL', y='FLUX.1',palette="Set2")
sns.boxplot(ax=axes[1], data=train_data, x='LABEL', y='FLUX.2',palette="Set2")
sns.boxplot(ax=axes[2], data=train_data, x='LABEL', y='FLUX.3',palette="Set2")
sns.boxplot(ax=axes[3], data=train_data, x='LABEL', y='FLUX.4',palette="Set2")
sns.boxplot(ax=axes[4], data=train_data, x='LABEL', y='FLUX.5',palette="Set2")

##### On affiche la moyenne de variation lumineuse des étoiles possédant une ou des exoplanète(s)

In [ ]:
train_data.loc[train_data['LABEL'] == 1].std(axis=1).mean()

##### On affiche la moyenne de variation lumineuse des étoiles ne possédant pas d'exoplanètes

In [ ]:
train_data.loc[train_data['LABEL'] == 0].std(axis=1).mean()

#### On observe que la variation lumineuse des étoiles possédant des exoplanètes est 2 fois plus importante

### On va split le dataset pour travailler dessus et pouvoir préparer le model

In [ ]:
X_train = train_data.drop(["LABEL"],axis=1)
y_train = train_data["LABEL"]   
X_test = test_data.drop(["LABEL"],axis=1)
y_test = test_data["LABEL"]

### Normalisation de la donnée

#### On va normaliser les données pour la préparer pour le machine learning. L'idée est d'organiser toutes les données dans un même fourchette pour faciliter l'apprentissage (tout en gardant les ordres de grandeur)

In [ ]:
# X_train = normalize(X_train)
# X_test = normalize(X_test)

### Feature scaling

In [ ]:
# std_scaler = StandardScaler()
# X_train = scaled = std_scaler.fit_transform(X_train)
# X_test = std_scaler.fit_transform(X_test)

# On définit les modèles qu'on va utiliser

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Test d'algorithme

### Decision Tree

In [ ]:
model_DT = DecisionTreeClassifier()

In [ ]:
model_DT.fit(X_train, y_train)
print(model_DT.__class__.__name__, model_DT.score(X_test, y_test))
print(confusion_matrix(y_test, model_DT.predict(X_test)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(y_test,model_DT.predict(X_test)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

### RandomForest

In [ ]:
model_RF = RandomForestClassifier()

In [ ]:
model_RF.fit(X_train, y_train)
print(model_RF.__class__.__name__, model_RF.score(X_test, y_test))
print(confusion_matrix(y_test, model_RF.predict(X_test)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(y_test,model_RF.predict(X_test)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

###  On arrive pas ou très difficilement à déterminer si une étoile possède des planètes

### Nous sommes en présent d'un accuracy paradox dû au déséquilibre des données

### Comme on l'a vu, l'échantillon est très déséquilibré. On va tenter de rééquilibrer les 2 catégories grâce à un SMOTE pour faciliter l'apprentissage.

## SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
smote = SMOTE(random_state=1)
train_X, train_y = smote.fit_sample(train_data.drop('LABEL',axis=1), train_data['LABEL'])

In [ ]:
train_X

In [ ]:
train_y

In [ ]:
plt.figure(figsize=(4,8))
colors = ["0", "1"]
sns.countplot(x=train_y, palette = "Set2")
plt.title('Stars Distribution \n (0: Etoiles sans exoplanètes || 1: Etoiles avec exoplanètes)', fontsize=14)

### On normalize et on scale

In [ ]:
train_X = normalize(train_X)

In [ ]:
std_scaler = StandardScaler()
train_X = scaled = std_scaler.fit_transform(train_X)

### On split les données smotée en train et validation

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=42)
# , stratify=train_y

### Maintenant qu'on a rééquilibré le data set on va pouvoir re-tester

## Decision Tree

In [ ]:
model_DT_SMOTE = DecisionTreeClassifier()

In [ ]:
model_DT_SMOTE.fit(train_X, train_y)
print(model_DT_SMOTE.__class__.__name__, model_DT_SMOTE.score(test_X, test_y))
print(confusion_matrix(test_y, model_DT_SMOTE.predict(test_X)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(test_y,model_DT_SMOTE.predict(test_X)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

## KNN

In [ ]:
model_KNN_SMOTE = KNeighborsClassifier()

In [ ]:
model_KNN_SMOTE.fit(train_X, train_y)
print(model_KNN_SMOTE.__class__.__name__, model_KNN_SMOTE.score(test_X, test_y))
print(confusion_matrix(test_y, model_KNN_SMOTE.predict(test_X)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(test_y,model_KNN_SMOTE.predict(test_X)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

## Random Forest

In [ ]:
model_RF_SMOTE = RandomForestClassifier()

In [ ]:
model_RF_SMOTE = model_RF_SMOTE.fit(train_X, train_y)
print(model_RF_SMOTE.__class__.__name__, model_RF_SMOTE.score(test_X, test_y))
print(confusion_matrix(test_y, model_RF_SMOTE.predict(test_X)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(test_y,model_RF_SMOTE.predict(test_X)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

## Logistic Regression

In [ ]:
model_LR_SMOTE = LogisticRegression()

In [ ]:
model_LR_SMOTE.fit(train_X, train_y)
print(model_LR_SMOTE.__class__.__name__, model_LR_SMOTE.score(test_X, test_y))
print(confusion_matrix(test_y, model_LR_SMOTE.predict(test_X)))

In [ ]:
#Confusion matrix
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(test_y,model_LR_SMOTE.predict(test_X)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

## Voting classifier

In [ ]:
#Retrait de l'ensemblevoting classifier car par interessant en terme de résultats

for model in (model_DT_SMOTE, model_KNN_SMOTE,model_RF_SMOTE, model_LR_SMOTE):
    model.fit(train_X, train_y)
    
    print(model.__class__.__name__)
    print('Accuracy score:',accuracy_score(y_test, model.predict(X_test)))
    print('ROCAUC score:',roc_auc_score(y_test, model.predict(X_test)))
    print('Precision score:',precision_score(y_test, model.predict(X_test)))
    print('Recall score:',recall_score(y_test, model.predict(X_test)))
    print('F1 score:',f1_score(y_test, model.predict(X_test)))
    print(confusion_matrix(y_test, model.predict(X_test)))
    print('')

## Tests de prédiction

In [ ]:
def prepare_data(df):

    if 'SAP_FLUX' in df.columns:
        df = transform_to_df(df)

    df = df.dropna(axis=1)
    
    
    if(df.size < 3197):
        print("Pas assez de donnéees")
        return None
    else:
        print("Assez de donnéees, traitement en cours ....")
                  
            
        df = remove_outliers(df)
        
        df_normalized = normalize(df)
        
        std_scaler = StandardScaler()
        df_normalized_scalled = scaled = std_scaler.fit_transform(df_normalized)
        
        df_transformed = pd.DataFrame(df_normalized_scalled) 
    
        if df_transformed.size > 3197:
            df_final = df_transformed[df_transformed.columns[0:3197]]
    
        print("Traitement terminé")
        return df_final

In [ ]:
def isexoplanet(model, df):
#     x = np.array(df[0]).reshape(1,3197)
    x = np.array(df).reshape(1, -1)
    
    print(model.predict(x))
    if model.predict(x) == 1:
        print("Cette étoile possède des exoplanètes")
    elif model.predict(x) == 0:
        print("Cette étoile ne possède pas d'exoplanètes")
        
    print(model.predict_proba(x))

In [ ]:
y_test[372:373]

In [ ]:
a = test_data[372:373].copy()
# show
a = a.T
a = a[1:-1]
ax2 = a.plot()

In [ ]:
isexoplanet(model_DT, X_test[372:373])

In [ ]:
y_pred = model_DT.predict(X_test)

In [ ]:
i=0

for i in range(len(y_pred)):
    if y_test[i] ==0 and y_pred[i].T == 1:
        print("False Positive : " + str(i))
    if y_test[i] == 1 and y_pred[i].T == 0 :
        print("False Negative  : " + str(i))
    i+=1

In [ ]:
plt.figure(figsize=(13,10))
plt.subplot(221)
sns.heatmap(confusion_matrix(y_test,model_DT.predict(X_test)),annot=True,cmap="viridis",fmt = "d",linecolor="k",linewidths=3)
plt.title("CONFUSION MATRIX",fontsize=20)

In [ ]:
print('ROCAUC score:',roc_auc_score(y_test, y_pred))
print('Accuracy score:',accuracy_score(y_test, y_pred))
print('Precision score:',precision_score(y_test, y_pred))
print('Recall score:',recall_score(y_test, y_pred))
print('F1 score:',f1_score(y_test, y_pred))